In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

#####สร้าง DataFrame หลัก ที่อ่านจากไฟล์ student.csv

In [ ]:
#สร้าง DataFrame หลัก ที่อ่านจากไฟล์ student.csv
df = pd.read_csv("./student.csv")

####ตรวจสอบว่าใน df มีค่า NaN อยู่กี่ช่องในแต่ละคอลัมน์

In [ ]:
df.isnull().sum()

####สร้าง DataFrame ใหม่ ชื่อdropna โดยคัดลอกข้อมูลจาก df เพื่อสร้างตัวสำเนาไว้ใช้งานต่อโดยไม่ไปแก้ไข df เดิม

In [ ]:
dropna = pd.DataFrame(df)

######แสดงข้อมูลก่อนลบ จะเห็นว่าข้อมูลที่ 10 อายุ เพศ คณะ ส่วนสูง ชั่วโมงเรียน จะเป็นค่า NaN จะใช้ ฟังก์ชั่น Dropna ลบแถวทั้งหมดที่มีค่า NaN ออกจาก DataFrame

In [ ]:
print("ก่อนลบ")
dropna

## ลบแถวที่มีค่าหาย

#####คืนค่า DataFrame ใหม่ ที่ลบทุกแถวที่มี NaN อยู่

In [ ]:
df_clean = df.dropna()

#####นับจำนวน NaN ว่ายังมีอยู่อีกหรือไม่

In [ ]:
df_clean.isnull().sum()

####สร้างตัวแปรมาชี้ไปที่ df

In [ ]:
df

####ข้อมูลก่อนแก้ไข

In [ ]:
df.isnull().sum()

##เติมค่าด้วยค่าเฉลี่ย

#####คำนวณค่าเฉลี่ยของคอลัมน์ 'อายุ''เกรดเฉลี่ย' 'ส่วนสูง' 'นํ้าหนัก' 'จำนวนชั่วโมงเรียน' โดยไม่นับรวมค่า NaN

In [ ]:
mean_age = df['อายุ'].mean()
mean_gpa = df['GPA'].mean()
mean_height = df['ส่วนสูง_cm'].mean()
mean_weight = df['น้ำหนัก_kg'].mean()
mean_hours=df['ชั่วโมงเรียน_ต่อสัปดาห์'].mean()

####แทนค่าที่ขาดหาย (NaN) ในแต่ละคอลัมน์ด้วยค่าเฉลี่ยของคอลัมน์นั้น และแก้ไขข้อมูลใน DataFrame เดิมโดยตรง ไม่สร้าง DataFrame ใหม่

In [ ]:
df['อายุ'].fillna(mean_age, inplace=True)
df['GPA'].fillna(mean_gpa, inplace=True)
df['ส่วนสูง_cm'].fillna(mean_height, inplace=True)
df['น้ำหนัก_kg'].fillna(mean_weight, inplace=True)
df['ชั่วโมงเรียน_ต่อสัปดาห์'].fillna(mean_hours, inplace=True)
df

##เติมค่าด้วยค่าที่พบบ่อยที่สุด

#####เติมค่าที่ขาดหายในคอลัมน์ประเภทข้อความ ด้วยค่าที่พบบ่อยที่สุดของแต่ละคอลัมน์

In [ ]:
df['คณะ'].fillna(df['คณะ'].mode()[0], inplace=True)
df['เพศ'].fillna(df['เพศ'].mode()[0], inplace=True)
df

##ทำให้ชื่อคณะ และเพศ เป็นมาตรฐานเดียวกัน

#####สร้างฟังก์ชันที่รับชื่อคณะเข้ามาแล้วแปลงเป็นชื่อมาตรฐาน

In [ ]:
def standardize_faculty(name):
#ตรวจสอบว่าชื่อที่รับเข้ามาอยู่ใน list ของชื่อที่เป็นไปได้หรือไม่
    if name in ['วิทยาศาสตร์', 'วิทย์', 'Science', 'Sci','คณะวิทยาศาสตร์']:
        return 'วิทยาศาสตร์' #คืนค่าชื่อคณะที่เป็นมาตรฐาน
    if name in ['วิศวกรรมศาสตร์','วิศวกรรม','วิศวฯ','วิศวะ','Engineering']:
       return 'วิศวกรรมศาสตร์'
    elif name in ['ศิลปศาสตร์', 'ศิลป์', 'Arts','Liberal Arts']:
        return 'ศิลปศาสตร์'#คืนค่าชื่อคณะที่เป็นมาตรฐาน
    else:
        return name #ถ้าไม่ตรงกับเงื่อนไขใดๆ ให้คืนชื่อเดิม

#####สร้างฟังก์ชันที่รับเพศเข้ามาแล้วแปลงเป็นชื่อมาตรฐาน

In [ ]:
def standardize_gender(name):
#ตรวจสอบว่าเพศที่รับเข้ามาอยู่ใน list ของชื่อที่เป็นไปได้หรือไม่
    if name in ['ชาย','Male','M','ช']:
      return 'ชาย' #คืนค่าที่เป็นมาตรฐาน
    elif name in ['หญิง','Female','F','ญ']:
      return 'หญิง' #คืนค่าที่เป็นมาตรฐาน
    else:
      return name #ถ้าไม่ตรงกับเงื่อนไขใดๆ ให้คืนค่าเดิม

#####ใช้ฟังก์ชันกับทุกค่าในคอลัมน์ 'คณะ' และสร้างคอลัมน์ใหม่ที่เก็บชื่อคณะที่เป็นมาตรฐานแล้ว

In [ ]:
df['คณะ'] = df['คณะ'].apply(standardize_faculty)

#####ใช้ฟังก์ชันกับทุกค่าในคอลัมน์ 'เพศ' และสร้างคอลัมน์ใหม่ที่เก็บค่าเพศที่เป็นมาตรฐานแล้ว

In [ ]:
df['เพศ'] = df['เพศ'].apply(standardize_gender)
df

##การจัดการค่าผิดปกติ

#####อายุ

In [ ]:
Q1_Age = df['อายุ'].quantile(0.25) #หา ควอไทล์ที่ 1 (Q1) ของข้อมูลอายุ หรือค่าที่อยู่ที่ 25% ของข้อมูลเมื่อเรียงจากน้อยไปมาก
Q3_Age = df['อายุ'].quantile(0.75) #หา ควอไทล์ที่ 3 (Q3) ของข้อมูลอายุ หรือค่าที่อยู่ที่ 75% ของข้อมูลเมื่อเรียงจากน้อยไปมาก
IQR_Age = Q3_Age - Q1_Age # Interquartile Range คือช่วงระหว่าง Q1 และ Q3 บอกถึงการกระจายตัวของข้อมูลส่วนกลาง 50%

#####ขอบเขตของ อายุ

In [ ]:
lower_bound_age = Q1_Age - 1.5 * IQR_Age  #กำหนดค่าต่ำสุด (lower_bound_age) และค่าสูงสุด (upper_bound_age) ตามสูตร IQR เพื่อใช้ระบุค่าอายุที่นอกขอบเขต (outliers)
upper_bound_age = Q3_Age + 1.5 * IQR_Age
print(lower_bound_age,upper_bound_age)

#####เก็บแถวที่ค่า 'อายุ' อยู่ในช่วง [lower_bound_age, upper_bound_age] เท่านั้น ตัด outlier ออก

In [ ]:
df = df[(df['อายุ'] >= lower_bound_age) & (df['อายุ'] <= upper_bound_age) & (df['GPA'] >= 0) & (df['GPA'] <= 4)]
df

#####ตรวจสอบว่าใน df ยังมีค่า NaN เหลืออยู่กี่ช่องในแต่ละคอลัมน์

In [ ]:
df.isnull().sum()

##คำนวณBMI

###### แปลงส่วนสูงจากเซนติเมตรเป็นเมตร (เพราะสูตร BMI ใช้หน่วยเมตร)

In [ ]:
df['BMI'] = df['น้ำหนัก_kg'] / (df['ส่วนสูง_cm']/100)**2
df['สุขภาพ'] = pd.cut(df['BMI'], #แบ่งค่า BMI เป็นช่วงตามเกณฑ์มาตรฐาน
                                   bins=[0, 18.5, 25, 30, 100],
                                   labels=['ผอม', 'ปกติ', 'อ้วน', 'อ้วนมาก'])

df